# The Bernstein–Vazirani Algorithm

## Objective
The **Bernstein–Vazirani algorithm** is a quantum algorithm designed to determine an unknown binary string  
$s \in \{0,1\}^n$ using **a single oracle query**.

The oracle hides the string $s$ inside a Boolean function defined as  
$f_s : \{0,1\}^n \rightarrow \{0,1\}$, where
$$
f_s(x) = s \cdot x \pmod{2} = \sum_{i=0}^{n-1} s_i x_i \pmod{2}.
$$
Here, $\cdot$ denotes the bitwise inner product modulo $2$.

---

## 1. State Preparation
We begin with:
- $n$ **input qubits**, each initialized to $|0\rangle$
- $1$ **target qubit**, initialized to $|1\rangle$

The initial state is
$$
|0\rangle^{\otimes n} \otimes |1\rangle.
$$

We apply a Hadamard gate to **all** qubits:
- $H^{\otimes n}$ to the input register
- $H$ to the target qubit

Using $H|1\rangle = |-\rangle = \frac{|0\rangle - |1\rangle}{\sqrt{2}}$, the resulting state becomes
$$
|\psi_1\rangle
= \left( \frac{1}{\sqrt{2^n}} \sum_{x \in \{0,1\}^n} |x\rangle \right) \otimes |-\rangle.
$$

This creates an equal superposition over all possible inputs $x$.

---

## 2. The Oracle and Phase Kickback
The oracle $U_f$ acts as
$$
U_f |x\rangle |y\rangle = |x\rangle |y \oplus f_s(x)\rangle.
$$

Since the target qubit is in the $|-\rangle$ state, the oracle does **not** change it directly. Instead, the function value $f_s(x)$ is encoded as a **phase** on the input register (this is known as *phase kickback*):
$$
|\psi_2\rangle
= \frac{1}{\sqrt{2^n}} \sum_{x \in \{0,1\}^n} (-1)^{f_s(x)} |x\rangle \otimes |-\rangle.
$$

Substituting $f_s(x) = s \cdot x$, we obtain
$$
|\psi_2\rangle
= \frac{1}{\sqrt{2^n}} \sum_{x \in \{0,1\}^n} (-1)^{s \cdot x} |x\rangle \otimes |-\rangle.
$$

The target qubit is now separable and can be ignored from further analysis.

---

## 3. Final Hadamard Transform
Next, we apply $H^{\otimes n}$ to the **input register**.

Using the identity
$$
H^{\otimes n} |x\rangle
= \frac{1}{\sqrt{2^n}} \sum_{z \in \{0,1\}^n} (-1)^{x \cdot z} |z\rangle,
$$
the resulting state becomes
$$
|\psi_3\rangle
= \frac{1}{2^n} \sum_{z \in \{0,1\}^n}
\sum_{x \in \{0,1\}^n}
(-1)^{s \cdot x + x \cdot z} |z\rangle.
$$

Combining the exponents gives
$$
|\psi_3\rangle
= \frac{1}{2^n} \sum_{z}
\sum_{x}
(-1)^{x \cdot (s \oplus z)} |z\rangle.
$$
### **Why the Plus Sign Becomes XOR ?**

In the exponent of $(-1)$, the **plus sign** `+` effectively acts as a **XOR operation**.  

Consider the logic:

- If both $x \cdot s$ and $x \cdot z$ are $1$ (odd), then:

$$
1 + 1 = 2
$$

- Since 

$$
(-1)^2 = 1
$$

this is equivalent to 

$$
(-1)^0 = 1
$$

- In **XOR logic**, 

$$
1 \oplus 1 = 0
$$

Hence, the addition in the exponent behaves exactly like XOR modulo 2:

$$
(-1)^{a + b} = (-1)^{a \oplus b}
$$

This is why the plus sign in the exponent is often written as XOR in quantum computing derivations.

---

## 4. Measurement and Interference
The inner sum
$$
\sum_{x \in \{0,1\}^n} (-1)^{x \cdot (s \oplus z)}
$$
determines the amplitude of each basis state $|z\rangle$.

- **If $z = s$**:  
  Then $s \oplus z = 0$, so $x \cdot (s \oplus z) = 0$ for all $x$.
  Each term equals $1$, and the sum becomes $2^n$.

- **If $z \neq s$**:  
  The terms cancel due to destructive interference, and the sum equals $0$.

Evaluating the amplitude for $|s\rangle$ explicitly:
$$
\text{Amplitude of } |s\rangle
= \frac{1}{2^n} \sum_{x=0}^{2^n-1} (-1)^{x \cdot s + s \cdot x}
$$
$$
= \frac{1}{2^n} \sum_{x=0}^{2^n-1} (-1)^{2(s \cdot x)}
= \frac{1}{2^n} \sum_{x=0}^{2^n-1} 1
= 1.
$$

- Here, $2 (s \cdot x)$ is always divisible by 2.  
- Therefore, modulo 2: $2 (s \cdot x) \mod 2 = 0$  

This gives:

$$
(-1)^{2 (s \cdot x)} = 1
$$

So the sum becomes:

$$
\frac{1}{2^n} \sum_{x=0}^{2^n-1} 1 = 1
$$

✅ Hence, the amplitude of $|s\rangle$ is **1**, meaning this state survives with certainty after interference.

---

## 5. Conclusion
The final quantum state of the input register is
$$
|\psi_3\rangle = |s\rangle.
$$

Since the amplitude of $|s\rangle$ is $1$, **a single measurement** of the input register yields the hidden string $s$ with **100% probability**.

This demonstrates the exponential advantage of the Bernstein–Vazirani algorithm over its classical counterpart, which would require $n$ oracle queries.


# Cirq

In [7]:
import cirq
import numpy as np

'''
Bernstein-Vazirani Algorithm Implementation

The Bernstein-Vazirani algorithm is designed to determine a hidden bit string 'a'
using a quantum oracle. The oracle implements the function f(x) = a · x (mod 2), where
'x' is the input bit string and '·' denotes the bitwise inner product.
The algorithm requires only a single query to the oracle to determine the hidden bit string,
whereas a classical approach would require multiple queries.

'''


def func_bit_pattern(num_qubits):
    """
    Create a random bit pattern for the oracle.

    :param num_qubits: Number of input qubits
    :return: List of 0s and 1s representing the hidden bit string
    """
    bit_pattern = [np.random.randint(0, 2) for _ in range(num_qubits)]
    print(f"Function bit pattern: {''.join([str(x) for x in bit_pattern])}")
    return bit_pattern

def oracle(input_qubits, target_qubit, circuit, num_qubits, bit_pattern):
    """
    Define the oracle for the Bernstein-Vazirani algorithm.

    :param input_qubits: List of input qubits
    :param target_qubit: Target qubit
    :param circuit: Cirq circuit
    :param num_qubits: Number of input qubits
    :param bit_pattern: Hidden bit string as a list of 0s and 1s
    :return: Circuit with oracle applied
    """
    for i in range(num_qubits):
        if bit_pattern[i] == 1:
            circuit.append(cirq.CNOT(input_qubits[i], target_qubit))
    return circuit

def BV_algorithm(num_qubits, bit_pattern):
    """
    Implements the Bernstein-Vazirani algorithm.

    :param num_qubits: Number of input qubits
    :param bit_pattern: Hidden bit string as a list of 0s and 1s
    """
    # Create qubits
    input_qubits = [cirq.LineQubit(i) for i in range(num_qubits)]
    target_qubit = cirq.LineQubit(num_qubits)

    # Create circuit
    circuit = cirq.Circuit()

    # Apply Hadamard to input qubits
    circuit.append([cirq.H(q) for q in input_qubits])

    # Prepare target qubit in |−> state
    circuit.append([cirq.X(target_qubit), cirq.H(target_qubit)])

    # Apply oracle
    circuit = oracle(input_qubits, target_qubit, circuit, num_qubits, bit_pattern)

    # Apply Hadamard to input qubits again
    circuit.append([cirq.H(q) for q in input_qubits])

    # Measure input qubits
    circuit.append(cirq.measure(*input_qubits, key='Z'))

    # Print circuit diagram
    print("Bernstein-Vazirani Circuit Diagram")
    print(circuit)

    # Simulate
    sim = cirq.Simulator()
    results = sim.run(circuit, repetitions=1000)
    
    # Convert to dictionary
    results_dict = dict(results.histogram(key='Z'))
    print("Measurement Results:")
    print(results_dict)

    # Convert keys to binary string for clarity
    results_binary = {f"{k:0{num_qubits}b}": v for k, v in results_dict.items()}
    print("Distribution of bit pattern output from Bernstein-Vazirani Algorithm:")
    print(results_binary)

def main(num_qubits=6, bit_pattern=None):
    if bit_pattern is None:
        bit_pattern = func_bit_pattern(num_qubits)
    BV_algorithm(num_qubits, bit_pattern)

if __name__ == '__main__':
    main()


Function bit pattern: 010101
Bernstein-Vazirani Circuit Diagram
0: ───H───H───────────────────M('Z')───
                              │
1: ───H───────@───H───────────M────────
              │               │
2: ───H───H───┼───────────────M────────
              │               │
3: ───H───────┼───@───H───────M────────
              │   │           │
4: ───H───H───┼───┼───────────M────────
              │   │           │
5: ───H───────┼───┼───@───H───M────────
              │   │   │
6: ───X───H───X───X───X────────────────
Measurement Results:
{21: 1000}
Distribution of bit pattern output from Bernstein-Vazirani Algorithm:
{'010101': 1000}


# Qiskit

In [28]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
import numpy as np

def func_bit_pattern(num_qubits):
    """
    Create a random bit pattern for the Bernstein-Vazirani oracle.

    :param num_qubits: Number of input qubits
    :return: List of 0s and 1s representing the hidden bit string
    """
    # Generate random 0/1 for each qubit
    bit_pattern = [np.random.randint(0, 2) for _ in range(num_qubits)]
    # Print the hidden bit string
    print(f"Function bit pattern: {''.join([str(x) for x in bit_pattern])}")
    return bit_pattern

def oracle(circuit, input_qubits, target_qubit, num_qubits, bit_pattern):
    """
    Define the oracle for the Bernstein-Vazirani algorithm.

    :param circuit: Qiskit QuantumCircuit
    :param input_qubits: List of input qubit indices
    :param target_qubit: Target qubit index
    :param num_qubits: Number of input qubits
    :param bit_pattern: Hidden bit string as a list of 0s and 1s
    :return: Circuit with oracle applied
    """
    # For each input qubit, apply CNOT to target if bit pattern is 1
    for i in range(num_qubits):
        if bit_pattern[i] == 1:
            circuit.cx(input_qubits[i], target_qubit)
    return circuit

def BV_algorithm(num_qubits, bit_pattern):
    """
    Implements the Bernstein-Vazirani algorithm.

    :param num_qubits: Number of input qubits
    :param bit_pattern: Hidden bit string as a list of 0s and 1s
    """
    # Total qubits = input qubits + 1 target qubit
    total_qubits = num_qubits + 1
    # Classical bits to store measurement results
    circuit = QuantumCircuit(total_qubits, num_qubits)

    # Input qubits indices
    input_qubits = list(range(num_qubits))
    # Target qubit index
    target_qubit = num_qubits

    # Step 1: Apply Hadamard gates to all input qubits
    for q in input_qubits:
        circuit.h(q)

    # Step 2: Prepare target qubit in |−> state
    circuit.x(target_qubit)
    circuit.h(target_qubit)

    # Step 3: Apply the oracle
    circuit = oracle(circuit, input_qubits, target_qubit, num_qubits, bit_pattern)

    # Step 4: Apply Hadamard gates to input qubits again
    for q in input_qubits:
        circuit.h(q)

    # Step 5: Measure input qubits
    # Reverse classical bit mapping to match original bit order
    for i, q in enumerate(input_qubits):
        circuit.measure(q, num_qubits - 1 - i)

    # Print circuit diagram
    print("Bernstein-Vazirani Circuit Diagram")
    print(circuit.draw('text'))

    # Step 6: Simulate the circuit
    simulator = AerSimulator()
    result = simulator.run(circuit, shots=1000).result()
    counts = result.get_counts()

    # Step 7: Print measurement results
    print("Measurement Results:")
    print(counts)

    # Step 8: Show distribution in proper binary order
    print("Distribution of bit pattern output from Bernstein-Vazirani Algorithm:")
    print(counts)

def main(num_qubits=6, bit_pattern=None):
    """
    Main function to run the Bernstein-Vazirani algorithm.

    :param num_qubits: Number of input qubits
    :param bit_pattern: Optional hidden bit string. Random if None
    """
    if bit_pattern is None:
        bit_pattern = func_bit_pattern(num_qubits)
    BV_algorithm(num_qubits, bit_pattern)

if __name__ == '__main__':
    main()


Function bit pattern: 100101
Bernstein-Vazirani Circuit Diagram
     ┌───┐          ┌───┐                ┌─┐           
q_0: ┤ H ├───────■──┤ H ├────────────────┤M├───────────
     ├───┤┌───┐  │  └┬─┬┘                └╥┘           
q_1: ┤ H ├┤ H ├──┼───┤M├──────────────────╫────────────
     ├───┤├───┤  │   └╥┘ ┌─┐              ║            
q_2: ┤ H ├┤ H ├──┼────╫──┤M├──────────────╫────────────
     ├───┤└───┘  │    ║  └╥┘        ┌───┐ ║      ┌─┐   
q_3: ┤ H ├───────┼────╫───╫──────■──┤ H ├─╫──────┤M├───
     ├───┤┌───┐  │    ║   ║ ┌─┐  │  └───┘ ║      └╥┘   
q_4: ┤ H ├┤ H ├──┼────╫───╫─┤M├──┼────────╫───────╫────
     ├───┤└───┘  │    ║   ║ └╥┘  │        ║ ┌───┐ ║ ┌─┐
q_5: ┤ H ├───────┼────╫───╫──╫───┼────■───╫─┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐  ║   ║  ║ ┌─┴─┐┌─┴─┐ ║ └───┘ ║ └╥┘
q_6: ┤ X ├┤ H ├┤ X ├──╫───╫──╫─┤ X ├┤ X ├─╫───────╫──╫─
     └───┘└───┘└───┘  ║   ║  ║ └───┘└───┘ ║       ║  ║ 
c: 6/═════════════════╩═══╩══╩════════════╩═══════╩══╩═
                      4   3  1          

### Example: Let The  Hidden Bit Pattern Be `100101`

- **Number of input qubits:** $n = 6$  
- **Hidden bit string (oracle):** $s = 100101$  
  - $s_0 = 1, s_1 = 0, s_2 = 0, s_3 = 1, s_4 = 0, s_5 = 1$  
- **Target qubit:** initialized in $|0\rangle$

---

### Step 1: Initial State

All qubits start in $|0\rangle$:

$$
|\psi_0\rangle = |000000\rangle |0\rangle
$$

Target qubit is later prepared in $|-\rangle = \frac{|0\rangle - |1\rangle}{\sqrt{2}}$.

---

### Step 2: Apply Hadamard to Input Qubits

Hadamard gates create a superposition of all possible input states:

$$
|\psi_1\rangle = \frac{1}{\sqrt{2^6}} \sum_{x=0}^{63} |x\rangle |0\rangle
$$

- Now all $2^6 = 64$ states of the input qubits are equally probable.

---

### Step 3: Prepare Target Qubit in $|-\rangle$

Apply $X$ then $H$ to target qubit:

$$
|0\rangle \xrightarrow{X} |1\rangle \xrightarrow{H} |-\rangle
$$

- Target qubit is ready to encode the oracle phase.

---

### Step 4: Apply Oracle

Oracle flips the target qubit based on the hidden bit string:

$$
U_f |x\rangle |y\rangle = |x\rangle |y \oplus (s \cdot x)\rangle
$$

- Where $s \cdot x = s_0 x_0 \oplus s_1 x_1 \oplus \dots \oplus s_5 x_5$  
- Target qubit is in $|-\rangle$, so flipping it multiplies the input state amplitude by $(-1)^{s \cdot x}$:

$$
|x\rangle |-\rangle \xrightarrow{U_f} (-1)^{s \cdot x} |x\rangle |-\rangle
$$

- Phase now **encodes the hidden string** in the input qubits.

---

### Step 5: Apply Hadamard to Input Qubits Again

- After a second Hadamard on each input qubit:

$$
H^{\otimes 6} \sum_x (-1)^{s \cdot x} |x\rangle = |s\rangle
$$

- All states except the hidden bit string $|100101\rangle$ cancel due to **destructive interference**.  
- Only the hidden bit string survives.

---

### Step 6: Measure Input Qubits

- Measure input qubits into classical bits (reversed order in Qiskit to match the bit pattern).  
- Result is **deterministic**:

- Measurement Results: ** {'100101': 1000} **
- Distribution of bit pattern output from Bernstein-Vazirani Algorithm: **{'100101': 1000}**


- ✅ The algorithm identifies the hidden bit string in **a single query**, unlike classical algorithms.

---

### Summary Table

| Step | Operation | State / Outcome |
|------|-----------|----------------|
| 0 | Initial state | $000000\rangle 0\rangle$ |
| 1 | Hadamard on input | Superposition of all 64 states |
| 2 | Prepare target qubit | $-$ |
| 3 | Oracle $U_f$ | Phase $(-1)^{s\cdot x}$ applied to input qubits |
| 4 | Hadamard on input | Only $100101$ survives |
| 5 | Measure | Deterministic result: `100101` |
